In [1]:
import pandas as pd

In [2]:
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker
import time
import secret


In [3]:
api_key = secret.api_key
api_secret = secret.api_secret

kite = KiteConnect(api_key=api_key)
print(kite.login_url())

https://kite.zerodha.com/connect/login?api_key=x5ugrx91yexerba1&v=3


In [4]:
data = kite.generate_session("zrzQdmXxFaLHgUJ0sbBd6SVW53zfB0En", api_secret=api_secret)
kite.set_access_token(data["access_token"])
print(data["access_token"])

wnxEcjVZlSfMz3NWiOpDObKPBPx5P95U


In [5]:
access_token = data["access_token"]
kws = KiteTicker(api_key, access_token)

In [6]:
def percentage(close_price,percent_value):
    percnt = (close_price/100)*percent_value
    addprcntge = close_price+percnt
    subprcntge = close_price-percnt
    return addprcntge,subprcntge
       

In [7]:
def swingTop_bottom(monthly_data):
    data=[]
    length = len(monthly_data)

# n = int(inpucollectiont())  this will be used for specifying the counter for swing top
    for i in range(2,length-2):
        if ((monthly_data['high'][i] > monthly_data['high'][i+1]) and (monthly_data['high'][i] > monthly_data['high'][i+2]) and (monthly_data['high'] [i]> monthly_data['high'][i-1]) and (monthly_data['high'][i] > monthly_data['high'][i-2])):
            data.append((float(monthly_data['high'][i]),str(pd.Period(monthly_data['date'][i]))))
         
        if ((monthly_data['low'][i] < monthly_data['low'][i+1]) and (monthly_data['low'][i] < monthly_data['low'][i+2]) and (monthly_data['low'][i] < monthly_data['low'][i-1]) and (monthly_data['low'][i] < monthly_data['low'][i-2])):
            data.append((float(monthly_data['low'][i]),str(pd.Period(monthly_data['date'][i]))))

    return data

In [8]:
def filterdTop(swingList,filteredata,levelAllowance):
    swingLen = len(swingList)
    filterdataLen = len(filteredata)
    # check=[]
    data ={}
    for i in range(swingLen):
        price =swingList[i]
        # print(price)
        percentage = ((price[0]/100)*levelAllowance)
        subPercentage = price[0] - percentage
        addPercentage = price[0] + percentage
        # print(subPercentage,addPercentage)
        key = price
        # check.append('END')
        if key not in data:
            data[key]=0
        for j in range(filterdataLen):
            if ((float(filteredata['open'][j]) <= addPercentage) and (float(filteredata['open'][j]) >= subPercentage)):
                data[key]+=1
                # check.append([(filteredata['date'][j]),'open',float(filteredata['open'][j])])
            elif ((float(filteredata['high'][j]) <= addPercentage) and (float(filteredata['high'][j]) >= subPercentage)):
                data[key]+=1
                # check.append([(filteredata['date'][j]),'high',float(filteredata['high'][j])])
            elif ((float(filteredata['low'][j]) <= addPercentage) and (float(filteredata['low'][j]) >= subPercentage)):
                data[key]+=1
                # check.append([(filteredata['date'][j]),'low',float(filteredata['low'][j])])
            elif ((float(filteredata['close'][j]) <= addPercentage) and (float(filteredata['close'][j]) >= subPercentage)):
                data[key]+=1
                # check.append([(filteredata['date'][j]),'close',float(filteredata['close'][j])])
    
    return data

In [9]:


def first_shortlist(symbol,addpercentage,subpercentage,ohlcCompare,close_data):
    stock_dict ={}
    stock_dict[symbol]=[set(),close_data]
    st = []
    for key in ohlcCompare.keys():
        level,period = eval(key)
        st.append((level,period))
    # print(st)
    close_add_percentage, close_sub_percentage = percentage(close_data, 4)
    for levels in range(len(st)):
       
        if (subpercentage <= float(st[levels][0]) <= addpercentage):
            
            if levels + 2 < len(st) and levels - 2 >= 0:
                stock_dict[symbol][0].add((st[levels],ohlcCompare[str(st[levels])])) 
                if levels + 2 < len(st) and close_sub_percentage <= float(st[levels + 2][0]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 2],ohlcCompare[str(st[levels + 2])])) 
                if levels + 1 < len(st) and close_sub_percentage <= float(st[levels + 1][0]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 1],ohlcCompare[str(st[levels + 1])])) 
                if levels - 1 >= 0 and close_sub_percentage <= float(st[levels - 1][0]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 1],ohlcCompare[str(st[levels -1])])) 
                if levels - 2 >= 0 and close_sub_percentage <= float(st[levels - 2][0]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 2],ohlcCompare[str(st[levels -2])]))
                
            elif levels + 2 < len(st): 
                stock_dict[symbol][0].add((st[levels],ohlcCompare[str(st[levels])]))
                if levels + 2 < len(st) and close_sub_percentage <= float(st[levels + 2][0]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 2],ohlcCompare[str(st[levels + 2])])) 
                if levels + 1 < len(st) and close_sub_percentage <= float(st[levels + 1][0]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 1],ohlcCompare[str(st[levels + 1])])) 
                 

            elif levels - 2 >= 0: 
                stock_dict[symbol][0].add((st[levels],ohlcCompare[str(st[levels])])) 
                if levels - 1 >= 0 and close_sub_percentage <= float(st[levels - 1][0]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 1],ohlcCompare[str(st[levels - 1])])) 
                if levels - 2 >= 0 and close_sub_percentage <= float(st[levels - 2][0]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 2],ohlcCompare[str(st[levels - 2])])) 

            elif levels - 1 >= 0 and levels + 2 < len(st): 
                stock_dict[symbol][0].add((st[levels],ohlcCompare[str(st[levels])]))
                if levels + 2 < len(st) and close_sub_percentage <= float(st[levels + 2][0]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 2],ohlcCompare[str(st[levels + 2])])) 
                if levels + 1 < len(st) and close_sub_percentage <= float(st[levels + 1][0]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 1],ohlcCompare[str(st[levels + 1])])) 
                if levels - 1 >= 0 and close_sub_percentage <= float(st[levels - 1][0]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 1],ohlcCompare[str(st[levels - 1])])) 
                 
                

            elif levels + 1 < len(st) and levels - 2 >= 0: 
                stock_dict[symbol][0].add((st[levels],ohlcCompare[str(st[levels])]))
                if levels + 1 < len(st) and close_sub_percentage <= float(st[levels + 1][0]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 1],ohlcCompare[str(st[levels + 1])])) 
                if levels - 1 >= 0 and close_sub_percentage <= float(st[levels - 1][0]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 1],ohlcCompare[str(st[levels - 1])])) 
                if levels - 2 >= 0 and close_sub_percentage <= float(st[levels - 2][0]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 2],ohlcCompare[str(st[levels - 2])]))
                

   
    if stock_dict[symbol][0]:
        return stock_dict
    
    

In [37]:
import datetime
import json
watchlist ="nifty500"
timeframe = 'M'
instruments = kite.instruments("NSE")
nifty50 =  pd.read_csv(f"E:\\TRADING KARNAL\\OHLCcompare\\watchList\\{watchlist}.csv")
errorStockList=[]
levelTapping={}

for symbols in nifty50['Symbol']:
    try:
        instrument_token = next(item["instrument_token"] for item in instruments if item["tradingsymbol"] == symbols)
        to_date = "2024-07-31"
        from_date = "2019-10-01"
        data = kite.historical_data(instrument_token, from_date, to_date, "day")
        df = pd.DataFrame(data)
        try:
            df['date'] = pd.to_datetime(df['date']).dt.to_period(timeframe) # WE MADE CHANGES FORM 'M' TO 'W' WHICH CONVERTED INTO WEEKS FROM MONTHS
            monthly_data = df.groupby('date').agg({
                'open': 'first',
                'high': 'max',
                'low': 'min',
                'close': 'last',
                'volume': 'sum'
            }).reset_index()

            st = swingTop_bottom(monthly_data)
            # print(st) # important levels for each stock
            ohlcCompare = filterdTop(st,df,levelAllowance=0.15)
            levelTapping[symbols]=ohlcCompare
        except:
            print(df)
            continue
    except StopIteration:
        errorStockList.append(symbols)
        continue
jasonfile = {k: json.dumps({str(key): value for key,value in v.items()}) for k,v in levelTapping.items()}
df= pd.DataFrame(list(jasonfile.items()),columns=["stocks","levels"])
df.to_csv("levelsTaps.csv",index=False)

C:\Users\91965\AppData\Local\Temp\ipykernel_13672\2759963098.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['date'] = pd.to_datetime(df['date']).dt.to_period(timeframe) # WE MADE CHANGES FORM 'M' TO 'W' WHICH CONVERTED INTO WEEKS FROM MONTHS
C:\Users\91965\AppData\Local\Temp\ipykernel_13672\2759963098.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['date'] = pd.to_datetime(df['date']).dt.to_period(timeframe) # WE MADE CHANGES FORM 'M' TO 'W' WHICH CONVERTED INTO WEEKS FROM MONTHS
C:\Users\91965\AppData\Local\Temp\ipykernel_13672\2759963098.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['date'] = pd.to_datetime(df['date']).dt.to_period(timeframe) # WE MADE CHANGES FORM 'M' TO 'W' WHICH CONVERTED INTO WEEKS FROM MONTHS
C:\Users\91965\AppData\Local\Temp\ipykernel_13672\2759963098.py:18: UserWarning: Converting to PeriodAr

Empty DataFrame
Columns: []
Index: []


C:\Users\91965\AppData\Local\Temp\ipykernel_13672\2759963098.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['date'] = pd.to_datetime(df['date']).dt.to_period(timeframe) # WE MADE CHANGES FORM 'M' TO 'W' WHICH CONVERTED INTO WEEKS FROM MONTHS
C:\Users\91965\AppData\Local\Temp\ipykernel_13672\2759963098.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['date'] = pd.to_datetime(df['date']).dt.to_period(timeframe) # WE MADE CHANGES FORM 'M' TO 'W' WHICH CONVERTED INTO WEEKS FROM MONTHS
C:\Users\91965\AppData\Local\Temp\ipykernel_13672\2759963098.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['date'] = pd.to_datetime(df['date']).dt.to_period(timeframe) # WE MADE CHANGES FORM 'M' TO 'W' WHICH CONVERTED INTO WEEKS FROM MONTHS
C:\Users\91965\AppData\Local\Temp\ipykernel_13672\2759963098.py:18: UserWarning: Converting to PeriodAr

In [38]:
from datetime import date
import json
import ast
import os
from_date= 20
from_month= "{:02d}".format(8)
from_year= 2024
to_date= 30
to_month= "{:02d}".format(8)
to_year= 2024
today = date.today()
instruments = kite.instruments("NSE")
levelAllowance = 1
# nifty50 =  pd.read_csv(f"E:\\TRADING KARNAL\\OHLCcompare\\watchList\\{watchlist}.csv")
# leveltap = pd.read_csv("E:\\TRADING KARNAL\\OHLCcompare\\levelsTaps.csv")
# try:
#     leveltap['levels'] = leveltap['levels'].apply(json.loads)
# except:
#     pass



for days in range(from_date,to_date+1):
    nifty50 =  pd.read_csv(f"E:\\TRADING KARNAL\\OHLCcompare\\watchList\\{watchlist}.csv")
    leveltap = pd.read_csv("E:\\TRADING KARNAL\\OHLCcompare\\levelsTaps.csv")
    try:
        leveltap['levels'] = leveltap['levels'].apply(json.loads)
        # print(days)
    except:
        pass
    i=0
    first_sort= {}
    levelTapping ={}
    errorStockList=[]
    for symbols in nifty50['Symbol']:
        try:
            instrument_token = next(item["instrument_token"] for item in instruments if item["tradingsymbol"] == symbols)
            # print(f"{to_year}-{to_month}-{days}")
            data =  kite.historical_data(instrument_token, f"{to_year}-{to_month}-{days}", f"{to_year}-{to_month}-{days}", "day")
            if data:
                close_data = data[0]['close']
                
                addpercentage,subpercentage = percentage(close_data,levelAllowance) #THIS VALUES REPRESENTS THE RANGE FROM LTP
                # print(type(leveltap.iloc[i]['levels']))
                if(leveltap.iloc[i]['levels']):
                    sortedlist = first_shortlist(symbols,addpercentage,subpercentage,leveltap.iloc[i]['levels'],close_data)
                    i+=1
                    if sortedlist != None:
                        first_sort[instrument_token] = sortedlist
                else:
                    # print(leveltap.iloc[i])
                    continue
        except StopIteration:
            errorStockList.append(symbols)
            continue

    flattened_data = []
    for key,values in first_sort.items():
        for symbol,prices in values.items():
            price_set =  list(prices[0])
            flattened_data.append([key,symbol,price_set,prices[1]])
    first_sorted_dataframe =  pd.DataFrame(flattened_data,columns=['token','symbol','levels','ltp'])
    first_sorted_csv = first_sorted_dataframe.to_csv("first_sorted_list_csv.csv",index=False)
    csvFile= pd.read_csv('E:\\TRADING KARNAL\\OHLCcompare\\first_sorted_list_csv.csv')
    csvFile['levels'] =csvFile['levels'].apply(ast.literal_eval)
    df = csvFile.explode('levels')
    df= pd.DataFrame(df)
    print(days)
    try:
        os.makedirs(f"E:\\TRADING KARNAL\\OHLCcompare\\backtest\\{to_month}")
        df.to_csv(f"E:\\TRADING KARNAL\\OHLCcompare\\backtest\\{to_month}\\{days}.csv",index=False)
    except FileExistsError:
        df.to_csv(f"E:\\TRADING KARNAL\\OHLCcompare\\backtest\\{to_month}\\{days}.csv",index=False)
    
    # try:
        # os.makedirs(f"E:\\TRADING KARNAL\\OHLCcompare\\backtest\\{to_month}")
    #     df.to_csv(f"E:\\TRADING KARNAL\\OHLCcompare\\backtest\\{to_month}\\{to_date}.csv",index=False)
    # except FileExistsError:
        
    
# if levelAllowance==1:
#     df.to_csv(f'tapList{watchlist}_{timeframe}_positional_{today.strftime("%d_%m_%Y")}.csv',index=False)
# else:
#     df.to_csv(f'tapList{watchlist}_{timeframe}_intraday_{today.strftime("%d_%m_%y")}_{today.strftime("%b_%d_%Y")}.csv',index=False)
    
# print(errorStockList)
# print(len(errorStockList))
# print(df)

20
21
22
23
24
25
26
27
28
29
30


In [39]:
import os
try:
    os.makedirs(f"E:\\TRADING KARNAL\\OHLCcompare\\backtest\\{to_month}")
except FileExistsError:
    
    print('error')


In [ ]:
# def on_connect(ws,response):
#     ws.subscribe(list(csvFile['token']))
#     ws.set_mode(ws.MODE_FULL, list(csvFile['token']))

# def on_close(ws,close,reason):
#     ws.stop()
#     # time.sleep(5) # Wait for 5 seconds before attempting to reconnect 
#     # kws.connect(threaded=True)
    

# kws.on_ticks = on_ticks 
# kws.on_connect = on_connect 
# kws.on_close = on_close 
# kws.connect(threaded=True)

# time.sleep(4)
# exit()


